# SIGNATURE VERIFICATION

In [4]:
########### THINNING ALGORITHM #############

from scipy import weave
import numpy as np
import cv2

def _thinningIteration(im, iter):
    I, M = im, np.zeros(im.shape, np.uint8)
    expr = """
    for (int i = 1; i < NI[0]-1; i++) {
        for (int j = 1; j < NI[1]-1; j++) {
            int p2 = I2(i-1, j);
            int p3 = I2(i-1, j+1);
            int p4 = I2(i, j+1);
            int p5 = I2(i+1, j+1);
            int p6 = I2(i+1, j);
            int p7 = I2(i+1, j-1);
            int p8 = I2(i, j-1);
            int p9 = I2(i-1, j-1);

            int A  = (p2 == 0 && p3 == 1) + (p3 == 0 && p4 == 1) +
                     (p4 == 0 && p5 == 1) + (p5 == 0 && p6 == 1) +
                     (p6 == 0 && p7 == 1) + (p7 == 0 && p8 == 1) +
            (p8 == 0 && p9 == 1) + (p9 == 0 && p2 == 1);
            int B  = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9;
            int m1 = iter == 0 ? (p2 * p4 * p6) : (p2 * p4 * p8);
            int m2 = iter == 0 ? (p4 * p6 * p8) : (p2 * p6 * p8);

            if (A == 1 && B >= 2 && B <= 6 && m1 == 0 && m2 == 0) {
                M2(i,j) = 1;
            }
        }
    } 
    """

    weave.inline(expr, ["I", "iter", "M"])
    return (I & ~M)


def thinning(src):
    dst = src.copy() / 255
    prev = np.zeros(src.shape[:2], np.uint8)
    diff = None

    while True:
        dst = _thinningIteration(dst, 0)
        dst = _thinningIteration(dst, 1)
        diff = np.absolute(dst - prev)
        prev = dst.copy()
        if np.sum(diff) == 0:
            break

    return dst * 255

############# ESCALAMIENTO #############
# Metodo que permite escalar la imagen a un tamaño por default(100)
# para que todas las imagenes posean el mismo tamaño
def scale(image, height=500):
    # Obtenemos las filas y columnas
    original_h, original_w = np.float32(image.shape[:2])
    # Calculamos el ancho que tendra con respecto al alto(100)
    width = int( height * ( original_w / original_h ) )
    # Escalamos la imagen
    image_scaled = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
    # Retornamos la imagen escalada
    return image_scaled

def normalize(image):
    # dilating image
    img = image.copy()
    kernel = np.ones((100,100), np.uint8)
    img = cv2.erode(img, kernel, iterations = 1)

    # Binarization
    ret,thresh = cv2.threshold(img,127,255,0)
    _,contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cnt = contours[0]

    # Bounding Rect
    x,y,w,h = cv2.boundingRect(cnt)

    # Cropping
    image = image[y:y+h,x:x+w]
    return image

############# PREPROCESSING #############
def preprocess(image, showProcess=False):
    #GrayScale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if showProcess:
        cv2.imshow(' GrayScale', image)
        cv2.waitKey(0)
        
    #Scaling 1
    image = scale(image)
    if showProcess:
        cv2.imshow(' Scaled 1', image)
        cv2.waitKey(0)

    # Normalizing
    image = normalize(image);
    if showProcess:
        cv2.imshow(' Normalized', image)
        cv2.waitKey(0)
        
    #Scaling 2
    image = scale(image)
    if showProcess:
        cv2.imshow(' Scaled 2', image)
        cv2.waitKey(0)

    # Binarizacion 1
    ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    if showProcess:
        cv2.imshow(' Binarizacion inv 1', image)
        cv2.waitKey(0)
        
    # Thinning
    image = thinning(image)
    if showProcess:
        cv2.imshow(' Thining', image)
        cv2.waitKey(0)
        
    # Binarizacion inv 2
    ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    if showProcess:
        cv2.imshow(' Binarizacion inv 2', image)
        cv2.waitKey(0)
    
    return image

def dilatex5(image):
    # DILATION 
    image_color_black = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image_color_red = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image_color_green= cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image_color_blue = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    image_color_black[image == 0] = [0, 0, 0]
    image_color_red[image == 0]   = [0, 0, 255]
    image_color_green[image == 0] = [0, 255, 0]
    image_color_blue[image == 0]  = [255, 0, 0]

    kernel_3 = np.ones((3,3), np.uint8)
    kernel_6 = np.ones((6,6), np.uint8)
    kernel_10 = np.ones((10,10), np.uint8) 
    kernel_16 = np.ones((16,16), np.uint8)

    image_3 = cv2.erode(image_color_black, kernel_3, iterations = 1)
    image_6 = cv2.erode(image_color_red, kernel_6, iterations = 1)
    image_10 = cv2.erode(image_color_green, kernel_10, iterations = 1)
    image_16 = cv2.erode(image_color_blue, kernel_16, iterations = 1)

    #cv2.imshow("Kernel 3", image_3)
    #cv2.imshow("Kernel 6", image_6)
    #cv2.imshow("Kernel 10", image_10)
    #cv2.imshow("Kernel 16", image_16)

    # FUSION
    img_fusion_1 = cv2.bitwise_and(image_16,image_10)
    img_fusion_1[cv2.threshold(cv2.cvtColor(img_fusion_1, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY)[1] == 0] = [0, 255, 0]

    img_fusion_2 = cv2.bitwise_and(img_fusion_1, image_6)
    img_fusion_2[cv2.threshold(cv2.cvtColor(img_fusion_2, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY)[1] == 0] = [0, 0, 255]

    img_fusion_3 = cv2.bitwise_and(img_fusion_2, image_3)

    return img_fusion_3


def obtenerFeatures(image):
    # GEOMETRIC FEATURES

    imHeight, imWidth = np.float32(image.shape[:2])
    totalPixels = imHeight*imWidth

    # Aspect Ratio
    aspectRatio = imHeight / imWidth
    print 'Aspect Ratio : ', aspectRatio

    # Density Ratio
    whitePixelsHalfLeft  = cv2.countNonZero(image[:,:image.shape[1]/2])
    signaturePixelsHalfLeft = totalPixels/2 - whitePixelsHalfLeft
    whitePixelsHalfRight = cv2.countNonZero(image[:,image.shape[1]/2:])
    signaturePixelsHalfRight = totalPixels/2 - whitePixelsHalfRight

    densityRatio = signaturePixelsHalfLeft / whitePixelsHalfRight

    print 'Density Ratio : ' , densityRatio

    # Ocupancy Ratio
    ## Negro 0 (cero)
    ## Otros(Blanco) NonZero
    totalWhitePixels = cv2.countNonZero(image)
    signaturePixels = totalPixels-totalWhitePixels
    occupancyRatio = signaturePixels / totalWhitePixels
    print 'Occupancy Ratio : ', occupancyRatio

    # Harris Corners / GoodFeatures to Track
    #cornersArr = cv2.cornerHarris(image, 3, 3, 0.05)
    cornersArr = cv2.goodFeaturesToTrack(image, 500, 0.01, 15)
    corners = len(cornersArr)
    print 'Corners : ' , corners
    
    # Imagen usada como base para comparar contra otras nuevas
    img_base = cv2.imread('images/signatures/drsc/legit_1_x5.jpg')
    img_entry = dilatex5(image)
    img_merged = cv2.bitwise_xor(img_base,img_entry)

    #cv2.imshow('im2',img_base)
    #cv2.imshow('im3',img_entry)
    #cv2.imshow('merged',img_merged)

    # NEGRO
    BLACK_MIN = np.array([0, 0, 0], np.uint8)
    BLACK_MAX = np.array([50, 50, 50], np.uint8)
    dst_black = cv2.inRange(img_merged, BLACK_MIN, BLACK_MAX)
    #cv2.imshow('Black',dst_black)
    no_black = cv2.countNonZero(dst_black)
    print('The number of black pixels is: ' + str(no_black))

    # ROJO
    RED_MIN = np.array([0, 0, 200], np.uint8)
    RED_MAX = np.array([50, 50, 255], np.uint8)
    dst_red = cv2.inRange(img_merged, RED_MIN, RED_MAX)
    #cv2.imshow('Red',dst_red)
    no_red = cv2.countNonZero(dst_red)
    print('The number of red pixels is: ' + str(no_red))

    # VERDE
    GREEN_MIN = np.array([0, 200, 0], np.uint8)
    GREEN_MAX = np.array([50, 255, 50], np.uint8)
    dst_green = cv2.inRange(img_merged, GREEN_MIN, GREEN_MAX)
    #cv2.imshow('Green',dst_green)
    no_green = cv2.countNonZero(dst_green)
    print('The number of green pixels is: ' + str(no_green))

    # AZUL
    BLUE_MIN = np.array([200, 0, 0], np.uint8)
    BLUE_MAX = np.array([255, 50, 50], np.uint8)
    dst_blue = cv2.inRange(img_merged, BLUE_MIN, BLUE_MAX)
    #cv2.imshow('Blue',dst_blue)
    no_blue = cv2.countNonZero(dst_blue)
    print('The number of blue pixels is: ' + str(no_blue))

    # BLANCO
    WHITE_MIN = np.array([200, 200, 200], np.uint8)
    WHITE_MAX = np.array([255, 255, 255], np.uint8)
    dst_white = cv2.inRange(img_merged, WHITE_MIN, WHITE_MAX)
    #cv2.imshow('White',dst_white)
    no_white = cv2.countNonZero(dst_white)
    print('The number of white pixels is: ' + str(no_white))

    # CIAN
    CIAN_MIN = np.array([200, 200, 0], np.uint8)
    CIAN_MAX = np.array([255, 255, 50], np.uint8)
    dst_cian = cv2.inRange(img_merged, CIAN_MIN, CIAN_MAX)
    #cv2.imshow('Cian',dst_cian)
    no_cian = cv2.countNonZero(dst_cian)
    print('The number of cian pixels is: ' + str(no_cian))

    # MAGENTA
    MAGENTA_MIN = np.array([200, 0, 200], np.uint8)
    MAGENTA_MAX = np.array([255, 50, 255], np.uint8)
    dst_magenta = cv2.inRange(img_merged, MAGENTA_MIN, MAGENTA_MAX)
    #cv2.imshow('Magenta',dst_magenta)
    no_magenta = cv2.countNonZero(dst_magenta)
    print('The number of magenta pixels is: ' + str(no_magenta))

    # AMARILLO
    YELLOW_MIN = np.array([0, 200, 200], np.uint8)
    YELLOW_MAX = np.array([50, 255, 255], np.uint8)
    dst_yellow= cv2.inRange(img_merged, YELLOW_MIN, YELLOW_MAX)
    #cv2.imshow('Yellow',dst_yellow)
    no_yellow = cv2.countNonZero(dst_yellow)
    print('The number of yellow pixels is: ' + str(no_yellow))

    # FONDO 1
    FONDO1_MIN = np.array([134, 130, 200], np.uint8)
    FONDO1_MAX = np.array([183, 180, 255], np.uint8)
    dst_fondo1= cv2.inRange(img_merged, FONDO1_MIN, FONDO1_MAX)
    #cv2.imshow('Fondo 1',dst_fondo1)
    no_fondo1 = cv2.countNonZero(dst_fondo1)
    print('The number of fondo1 pixels is: ' + str(no_fondo1))

    # FONDO 2
    FONDO2_MIN = np.array([71, 75, 0], np.uint8)
    FONDO2_MAX = np.array([121, 125, 50], np.uint8)
    dst_fondo2= cv2.inRange(img_merged, FONDO2_MIN, FONDO2_MAX)
    #cv2.imshow('Fondo 2',dst_fondo2)
    no_fondo2 = cv2.countNonZero(dst_fondo2)
    print('The number of fondo2 pixels is: ' + str(no_fondo2))
    
    featuresX = [
        aspectRatio,
        densityRatio,
        occupancyRatio,
        corners,
        no_red,
        no_green,
        no_blue,
        no_white,
        no_cian,
        no_magenta,
        no_yellow,
        no_fondo1,
        no_fondo2 ]
    return featuresX
    

In [9]:
##### TESTING GENERAL ########

import cv2
import numpy as np
ruta_imagen = "images/signatures/drsc/legit_5"
extension_imagen = ".jpg"
image = cv2.imread(ruta_imagen + extension_imagen)
image = preprocess(image, False)
cv2.imshow('Signature PreProcessed', image)
cv2.waitKey(0)

img_dilated = dilatex5(image)
cv2.imshow('Signature Dilated x5', img_dilated)
#cv2.imwrite(ruta_imagen+"_x5"+extension_imagen, img_dilated)
cv2.waitKey(0)

featuresX = obtenerFeatures(image)
featuresY = [1]

print "Caracteristicas : ", featuresX

cv2.destroyAllWindows()


Aspect Ratio :  0.56243
Density Ratio :  0.00773559318979
Occupancy Ratio :  0.00453339480309
Corners :  73
The number of black pixels is: 406951
The number of red pixels is: 1394
The number of green pixels is: 1059
The number of blue pixels is: 1500
The number of white pixels is: 3409
The number of cian pixels is: 4461
The number of magenta pixels is: 4813
The number of yellow pixels is: 7210
The number of fondo1 pixels is: 21
The number of fondo2 pixels is: 20
Caracteristicas :  [0.56242967, 0.0077355931897946035, 0.0045333948030933753, 73, 1394, 1059, 1500, 3409, 4461, 4813, 7210, 21, 20]


In [3]:
# Testeando la dilatacion de colores
import cv2
import numpy as np

im2 = cv2.imread('images/signatures/drsc/legit_1_x5.jpg')
im3 = cv2.imread('images/signatures/drsc/legit_2_x5.jpg')

im_merged = cv2.bitwise_xor(im2,im3)

cv2.imshow('im2',im2)
cv2.imshow('im3',im3)
cv2.imshow('merged',im_merged)

# NEGRO
BLACK_MIN = np.array([0, 0, 0], np.uint8)
BLACK_MAX = np.array([50, 50, 50], np.uint8)
dst_black = cv2.inRange(im_merged, BLACK_MIN, BLACK_MAX)
cv2.imshow('Black',dst_black)
no_black = cv2.countNonZero(dst_black)
print('The number of black pixels is: ' + str(no_black))

# ROJO
RED_MIN = np.array([0, 0, 200], np.uint8)
RED_MAX = np.array([50, 50, 255], np.uint8)
dst_red = cv2.inRange(im_merged, RED_MIN, RED_MAX)
cv2.imshow('Red',dst_red)
no_red = cv2.countNonZero(dst_red)
print('The number of red pixels is: ' + str(no_red))

# VERDE
GREEN_MIN = np.array([0, 200, 0], np.uint8)
GREEN_MAX = np.array([50, 255, 50], np.uint8)
dst_green = cv2.inRange(im_merged, GREEN_MIN, GREEN_MAX)
cv2.imshow('Green',dst_green)
no_green = cv2.countNonZero(dst_green)
print('The number of green pixels is: ' + str(no_green))

# AZUL
BLUE_MIN = np.array([200, 0, 0], np.uint8)
BLUE_MAX = np.array([255, 50, 50], np.uint8)
dst_blue = cv2.inRange(im_merged, BLUE_MIN, BLUE_MAX)
cv2.imshow('Blue',dst_blue)
no_blue = cv2.countNonZero(dst_blue)
print('The number of blue pixels is: ' + str(no_blue))

# BLANCO
WHITE_MIN = np.array([200, 200, 200], np.uint8)
WHITE_MAX = np.array([255, 255, 255], np.uint8)
dst_white = cv2.inRange(im_merged, WHITE_MIN, WHITE_MAX)
cv2.imshow('White',dst_white)
no_white = cv2.countNonZero(dst_white)
print('The number of white pixels is: ' + str(no_white))

# CIAN
CIAN_MIN = np.array([200, 200, 0], np.uint8)
CIAN_MAX = np.array([255, 255, 50], np.uint8)
dst_cian = cv2.inRange(im_merged, CIAN_MIN, CIAN_MAX)
cv2.imshow('Cian',dst_cian)
no_cian = cv2.countNonZero(dst_cian)
print('The number of cian pixels is: ' + str(no_cian))

# MAGENTA
MAGENTA_MIN = np.array([200, 0, 200], np.uint8)
MAGENTA_MAX = np.array([255, 50, 255], np.uint8)
dst_magenta = cv2.inRange(im_merged, MAGENTA_MIN, MAGENTA_MAX)
cv2.imshow('Magenta',dst_magenta)
no_magenta = cv2.countNonZero(dst_magenta)
print('The number of magenta pixels is: ' + str(no_magenta))

# AMARILLO
YELLOW_MIN = np.array([0, 200, 200], np.uint8)
YELLOW_MAX = np.array([50, 255, 255], np.uint8)
dst_yellow= cv2.inRange(im_merged, YELLOW_MIN, YELLOW_MAX)
cv2.imshow('Yellow',dst_yellow)
no_yellow = cv2.countNonZero(dst_yellow)
print('The number of yellow pixels is: ' + str(no_yellow))

# FONDO 1
FONDO1_MIN = np.array([134, 130, 200], np.uint8)
FONDO1_MAX = np.array([183, 180, 255], np.uint8)
dst_fondo1= cv2.inRange(im_merged, FONDO1_MIN, FONDO1_MAX)
cv2.imshow('Fondo 1',dst_fondo1)
no_fondo1 = cv2.countNonZero(dst_fondo1)
print('The number of fondo1 pixels is: ' + str(no_fondo1))

# FONDO 2
FONDO2_MIN = np.array([71, 75, 0], np.uint8)
FONDO2_MAX = np.array([121, 125, 50], np.uint8)
dst_fondo2= cv2.inRange(im_merged, FONDO2_MIN, FONDO2_MAX)
cv2.imshow('Fondo 2',dst_fondo2)
no_fondo2 = cv2.countNonZero(dst_fondo2)
print('The number of fondo2 pixels is: ' + str(no_fondo2))

cv2.waitKey(0)
cv2.destroyAllWindows()

The number of black pixels is: 406046
The number of red pixels is: 833
The number of green pixels is: 679
The number of blue pixels is: 1114
The number of white pixels is: 2990
The number of cian pixels is: 2733
The number of magenta pixels is: 2710
The number of yellow pixels is: 5376
The number of fondo1 pixels is: 44
The number of fondo2 pixels is: 22


In [19]:
# Vector de Caracteristicas
featuresX = [
    aspectRatio,
    densityRatio,
    occupancyRatio,
    corners,
    no_red,
    no_green,
    no_blue,
    no_white,
    no_cian,
    no_magenta,
    no_yellow,
    no_fondo1,
    no_fondo2
]

featuresY = [1]

print "Features X ", featuresX
print "Features Y ", featuresY

Features X  [0.56242967, 0.011948956439761292, 0.0072649979378826817, 500, 995, 839, 1410, 7316, 5593, 5306, 11714, 144, 36]
Features Y  [1]


In [64]:
##### TESTING PREPROCESSING ########
import cv2
import numpy as np

image = cv2.imread('images/signature.jpg')
cv2.imshow('1.Signature Original', image)
cv2.waitKey(0)
# A ESCALA DE GRISES
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('2.Signature GrayScale', image)
cv2.waitKey(0)

# ESCALAMIENTO
image = scale(image)
cv2.imshow('3.Signature Scaled', image)
cv2.waitKey(0)

# ELIMINACION DE RUIDO
image = cv2.fastNlMeansDenoising(image, None, 6, 7, 21)
cv2.imshow('4.Signature Denoised', image)
cv2.waitKey(0)

# BINARIZACION <127 -> 0 (black), >=127 -> 255 (white)
ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('5. Threshold Binary', image)
cv2.waitKey(0)

# THINING
image = thinning(image)
cv2.imshow('5. Thinning', image)
cv2.waitKey(0)


# *** ROTACION ***

# *** EROSION ***
#kernel = np.ones((5,5), np.uint8)
#image = cv2.dilate(image, kernel, iterations = 1)
#cv2.imshow("7. Erosion", image)
#cv2.waitKey(0)

# *** DILATACION ***
#kernel = np.ones((5,5), np.uint8)
#image = cv2.erode(image, kernel, iterations = 1)
#cv2.imshow("6. Dilation", image)
#cv2.waitKey(0)

cv2.destroyAllWindows()

In [10]:
# Contours - Para delimitar la firma
import cv2
import numpy as np

img = cv2.imread('images/signatures/drsc/legit_1.jpg',0)
img = scale(img)
cv2.imshow('Original',img)
cv2.waitKey(0)

# dilating image
kernel = np.ones((100,100), np.uint8)
img = cv2.erode(img, kernel, iterations = 1)

ret,thresh = cv2.threshold(img,127,255,0)
_,contours,hierarchy = cv2.findContours(thresh, 1, 2)

# Moments
cnt = contours[0]
M = cv2.moments(cnt)
print M

#Centroid
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
print 'contour area :', M['m00'] , cv2.contourArea(cnt)
print 'Contour perimeter : ', cv2.arcLength(cnt,True)
print 'centroid : ',cx,cy

# Contour Approximation
epsilon = 0.1*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)
# Hull
hull = cv2.convexHull(cnt)
# Bounding Rect
x,y,w,h = cv2.boundingRect(cnt)

cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
img = img[y:y+h,x:x+w]

cv2.imshow('Cropping',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

{'mu02': 792088982.9336376, 'mu03': 772342436.6479492, 'm11': 14776173335.125, 'nu02': 0.055520862367176155, 'm12': 4465346985876.434, 'mu21': 1255415229.5533447, 'mu20': 1984872112.2666054, 'nu20': 0.13912806987094944, 'm30': 13071571236107.8, 'nu21': 0.0002546186321410142, 'mu11': 64370774.86909294, 'mu12': -10017063627.520874, 'nu11': 0.004512019493995183, 'nu12': -0.0020316234651831535, 'm02': 10016970852.416666, 'm03': 3224823781108.5503, 'm00': 119442.5, 'm01': 33194019.833333332, 'mu30': 33796113766.69336, 'nu30': 0.006854401680326211, 'nu03': 0.00015664361092204086, 'm10': 52937682.33333333, 'm20': 25447192566.25, 'm21': 7130291512179.1}
contour area : 119442.5 119442.5
Contour perimeter :  1918.23253584
centroid :  443 277


In [7]:
cv2.imshow('half 1', image[:,:image.shape[1]/2])
cv2.imshow('half 2', image[:,image.shape[1]/2:])
cv2.waitKey(0)
cv2.destroyAllWindows()